In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from aif360.datasets import BinaryLabelDataset, AdultDataset

In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from functions import*

In [5]:
import os
path=os.path.dirname(os.getcwd())

In [6]:
class ROCpostprocess:
    def __init__(self,X_val,y_val,var_list,prediction_model,favorable_label):
        self.X_val =X_val
        self.y_val =y_val
        self.model = prediction_model
        self.positive_index = 1 # positive label
        self.var_list = var_list
        self.var_dim=len(self.var_list)
        self.ROC = self.buildROCusingval()
        self.favorable_label = favorable_label

    def buildbinarydata(self,X,y):
        df=pd.DataFrame(np.concatenate((X,y.reshape(-1,1)), axis=1),columns=self.var_list+['S','W','Y'])
        binaryLabelDataset = BinaryLabelDataset(
                            # favorable_label=self.favorable_label,
                            # unfavorable_label=0,
                            df=df[self.var_list+['S','W','Y']], #df_test.drop('X',axis=1), #[x_list+['S','W','Y']],
                            label_names=['Y'],
                            instance_weights_name=['W'],
                            protected_attribute_names=['S'],
                            privileged_protected_attributes=[np.array([1.0])],
                            unprivileged_protected_attributes=[np.array([0.])])
        return binaryLabelDataset,df

    def buildROCusingval(self):
        dataset_val = self.buildbinarydata(self.X_val,self.y_val)[0]
        dataset_val_pred = dataset_val.copy(deepcopy=True)
        dataset_val_pred.scores = self.model.predict_proba(dataset_val.features[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        privileged_groups = [{'S': 1}]
        unprivileged_groups = [{'S': 0}]
        # Metric used (should be one of allowed_metrics)
        metric_name = "Statistical parity difference"
        # Upper and lower bound on the fairness metric used
        metric_ub = 0.05
        metric_lb = -0.05
        ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                        privileged_groups=privileged_groups, 
                                        low_class_thresh=0.01, high_class_thresh=0.99,
                                        num_class_thresh=50, num_ROC_margin=10,
                                        metric_name=metric_name,
                                        metric_ub=metric_ub, metric_lb=metric_lb)
        ROC = ROC.fit(dataset_val, dataset_val_pred)
        print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
        print("Optimal ROC margin = %.4f" % ROC.ROC_margin)
        return ROC

    def postprocess(self,X_test,y_test,tv_origin): # the tv distance won't change
        dataset_test_pred,df_test = self.buildbinarydata(X_test,y_test) #.copy(deepcopy=True)
        dataset_test_pred.scores = self.model.predict_proba(X_test[:,0:self.var_dim])[:,self.positive_index].reshape(-1,1)
        dataset_test_pred_transf = self.ROC.predict(dataset_test_pred)
        y_pred = dataset_test_pred_transf.labels
        # return dataset_test_pred_transf.convert_to_dataframe()[0]

        di = DisparateImpact_postprocess(df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(df_test['Y'], y_pred, average='macro',sample_weight=df_test['W'])
        f1_micro = f1_score(df_test['Y'], y_pred, average='micro',sample_weight=df_test['W'])
        f1_weighted = f1_score(df_test['Y'], y_pred, average='weighted',sample_weight=df_test['W'])
        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv_origin,'method':'ROC'})
        return new_row.to_frame().T

In [7]:
class Projpostprocess:
    
    def __init__(self,X_test,y_test,x_list,var_list,prediction_model,K,e,thresh,favorable_label=1):
        self.model = prediction_model
        self.K=K
        self.e=e
        self.x_list=x_list
        self.var_list=var_list
        self.var_dim=len(var_list)
        self.favorable_label = favorable_label

        df_test=pd.DataFrame(np.concatenate((X_test,y_test.reshape(-1,1)), axis=1),columns=var_list+['S','W','Y'])
        df_test=df_test.groupby(by=var_list+['S','Y'],as_index=False).sum()
        if len(x_list)>1:
            df_test['X'] = list(zip(*[df_test[c] for c in x_list]))
            self.x_range=sorted(set(df_test['X']))
            weight=list(1/(df_test[x_list].max()-df_test[x_list].min())) # because 'education-num' range from 1 to 16 while others 1 to 4
            self.C=c_generate_higher(self.x_range,weight)
        else:
            df_test['X']=df_test[x_list]
            self.x_range=sorted(set(df_test['X']))
            self.C=c_generate(self.x_range)
        self.df_test = df_test
        self.var_range=list(pd.pivot_table(df_test,index=var_list,values=['S','W','Y']).index)
        self.distribution_generator()
        
        if thresh == 'auto':
            self.thresh_generator()
        else:
            self.thresh=thresh

    def distribution_generator(self):
        bin=len(self.x_range)
        dist=rdata_analysis(self.df_test,self.x_range,'X')
        
        dist['v']=[(dist['x_0'][i]-dist['x_1'][i])/dist['x'][i] for i in range(bin)]
        
        dist['t_x']=dist['x'] # #dist['x'] #dist['x_0']*0.5+dist['x_1']*0.5 
        self.px=np.matrix(dist['x']).T
        self.ptx=np.matrix(dist['t_x']).T
        if np.any(dist['x_0']==0): 
            self.p0=np.matrix((dist['x_0']+1.0e-9)/sum(dist['x_0']+1.0e-9)).T
        else:
            self.p0=np.matrix(dist['x_0']).T 
        if np.any(dist['x_1']==0):
            self.p1=np.matrix((dist['x_1']+1.0e-9)/sum(dist['x_1']+1.0e-9)).T
        else:
            self.p1=np.matrix(dist['x_1']).T 
        self.V=np.matrix(dist['v']).T
        self.tv_origin=sum(abs(dist['x_0']-dist['x_1']))/2
        # return px,ptx,V,p0,p1
    
    def coupling_generator(self,method,para=None):
        if method == 'unconstrained':
            coupling=baseline(self.C,self.e,self.px,self.ptx,self.K)
        elif method == 'barycentre':
            coupling=baseline(self.C,self.e,self.p0,self.p1,self.K)
        elif method == 'partial':
            # coupling = np.matrix(np.outer(self.px,self.ptx))
            coupling=partial_repair(self.C,self.e,self.px,self.ptx,self.V,para,self.K)
        return coupling

    def postprocess(self,method,para=None):
        if method == 'origin':
            y_pred=self.model.predict(np.array(self.df_test[self.var_list]))
            tv = self.tv_origin
        else:
            if (method == 'unconstrained') or (method == 'partial'):
                coupling = self.coupling_generator(method,para)
                y_pred=postprocess(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,self.thresh)
                tv=assess_tv(self.df_test,coupling,self.x_range,self.x_list,self.var_list)
                if (para != None) and (method == 'partial'):
                    method = method+'_'+str(para)
            elif method == 'barycentre':
                coupling = self.coupling_generator(method,para)
                y_pred,tv=postprocess_bary(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,self.thresh)
            else:
                print('Unknown method')

        di = DisparateImpact_postprocess(self.df_test,y_pred,favorable_label=self.favorable_label)
        f1_macro = f1_score(self.df_test['Y'], y_pred, average='macro',sample_weight=self.df_test['W'])
        f1_micro = f1_score(self.df_test['Y'], y_pred, average='micro',sample_weight=self.df_test['W'])
        f1_weighted = f1_score(self.df_test['Y'], y_pred, average='weighted',sample_weight=self.df_test['W'])

        new_row=pd.Series({'DI':di,'f1 macro':f1_macro,'f1 micro':f1_micro,'f1 weighted':f1_weighted,
                           'TV distance':tv,'method':method})
        return new_row.to_frame().T
    
    def thresh_generator(self):
        num_thresh = 10
        ba_arr = np.zeros(num_thresh)
        ba_arr1 = np.zeros(num_thresh)
        class_thresh_arr = np.linspace(0.1, 0.9, num_thresh)
        coupling=self.coupling_generator('partial',para=1e-3)
    
        for idx, thresh in enumerate(class_thresh_arr):
            y_pred=postprocess(self.df_test,coupling,self.x_list,self.x_range,self.var_list,self.var_range,self.model,thresh)
            ba_arr[idx] = DisparateImpact_postprocess(self.df_test,y_pred,favorable_label=self.favorable_label)
            ba_arr1[idx] = f1_score(self.df_test['Y'], y_pred, average='macro',sample_weight=self.df_test['W'])

        best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
        best_thresh = class_thresh_arr[best_ind]
        print("Optional threshold = ",class_thresh_arr)
        print("Disparate Impact = ",ba_arr)
        print("f1 scores = ",ba_arr1)
        self.thresh = best_thresh

In [8]:
def load_data(data_path,var_list,pa):
    column_names = ['age', 'workclass', 'fnlwgt', 'education',
                'education-num', 'marital-status', 'occupation', 'relationship',
                'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
                'native-country', 'Y']
    na_values=['?']
    pa_dict={'Male':1,'Female':0,'White':1,'Black':0}
    label_dict={'>50K.':1,'>50K':1,'<=50K.':0,'<=50K':0}
    train_path = os.path.join(data_path, 'adult.data')
    test_path = os.path.join(data_path, 'adult.test')
    train = pd.read_csv(train_path, header=None,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    test = pd.read_csv(test_path, header=0,names=column_names,
                    skipinitialspace=True, na_values=na_values)
    messydata = pd.concat([test, train], ignore_index=True)[var_list+[pa,'Y']]
    messydata=messydata.rename(columns={pa:'S'})
    messydata['S']=messydata['S'].replace(pa_dict)
    messydata['Y']=messydata['Y'].replace(label_dict)
    messydata=messydata[(messydata['S']==0)|(messydata['S']==1)]
    for col in var_list+['S','Y']:
        messydata[col]=messydata[col].astype('int64')
    messydata['W']=1
    bins_capitalgain=[100,3500,7500,10000]
    bins_capitalloss=[100,1600,1900,2200]
    bins_age=[26,36,46,56]
    bins_hours=[21,36,46,61]

    messydata=categerise(messydata,'age',bins_age)
    # messydata=categerise(messydata,'hours-per-week',bins_hours)
    messydata=categerise(messydata,'capital-gain',bins_capitalgain)
    messydata=categerise(messydata,'capital-loss',bins_capitalloss)
    
    return messydata

def categerise(df,col,bins):
    for i in range(len(bins)+1):
        if i == 0:
            df.loc[df[col] < bins[i], col] = i
        elif i == len(bins):
            df.loc[df[col] >= bins[i-1], col] = i
        else:
            df.loc[(df[col] >= bins[i-1])& (df[col] < bins[i]), col] = i        
    return df

def choose_x(var_list,messydata):
    tv_dist=dict()
    for x_name in var_list:
        x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
        dist=rdata_analysis(messydata,x_range_single,x_name)
        tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
    x_list=[]
    for key,val in tv_dist.items():
        if val>0.1:
            x_list+=[key]  
    return x_list,tv_dist

In [9]:
data_path='C://Users//zhouq//anaconda3//Lib//site-packages//aif360//data//raw//adult'
var_list=['age','capital-gain','capital-loss','education-num'] #'hours-per-week',
pa='sex'
favorable_label = 1
var_dim=len(var_list)

K=200
e=0.01

if pa == 'sex':
    thresh=0.05
elif pa == 'race':
    thresh=0.1

messydata = load_data(data_path,var_list,pa)
x_list,tv_dist = choose_x(var_list,messydata)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]

In [10]:
tv_dist

{'age': 0.1010227688866829,
 'capital-gain': 0.036924675713792855,
 'capital-loss': 0.020068855964263464,
 'education-num': 0.07095473385227195}

In [12]:
thresh=0.05
x_list = ['age','education-num']
methods=['origin','barycentre','partial']
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)
    
    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label)
    for method in methods[:-1]:
        report = pd.concat([report,projpost.postprocess(method)], ignore_index=True)
    
    for p in [1e-2,1e-3,1e-4]:
        report = pd.concat([report,projpost.postprocess('partial',para=p)], ignore_index=True)

report.to_csv(path+'/data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

c:\Users\zhouq\Documents\OT_Debiasing\E2\functions.py:166: RuntimeWarning: overflow encountered in exp
  fun = lambda z: sum(tmp.item(i,j)*V.item(i)*np.exp(-z*V.item(i)) for i in I)+theta.item(j)
c:\Users\zhouq\Documents\OT_Debiasing\E2\functions.py:167: RuntimeWarning: overflow encountered in exp
  dfun = lambda z: -sum(tmp.item(i,j)*(V.item(i))**2*np.exp(-z*V.item(i)) for i in I)
c:\Users\zhouq\Documents\OT_Debiasing\E2\functions.py:34: RuntimeWarning: invalid value encountered in scalar divide
  b=a-fun(a)/dfun(a)
c:\Users\zhouq\Documents\OT_Debiasing\E2\functions.py:198: RuntimeWarning: invalid value encountered in divide
  rdist['x']= np.array([pivot[i] for i in x_range])/sum([pivot[i] for i in x_range]) #empirical_distribution(rdata,x_range)
c:\Users\zhouq\Documents\OT_Debiasing\E2\functions.py:201: RuntimeWarning: invalid value encountered in divide
  rdist['x_0']=np.array([pivot0[i] if i in list(pivot0.index) else 0 for i in x_range])/sum([pivot0[i] if i in list(pivot0.index) e

In [13]:
report.dropna()

,DI,f1 macro,f1 micro,f1 weighted,TV distance,method
0,0.560671,0.677944,0.811537,0.784761,0.122976,origin
1,0.973587,0.576042,0.676306,0.682216,0.001136,barycentre
2,1.064175,0.598993,0.727645,0.715961,0.087166,partial_0.01
3,1.113045,0.578018,0.697548,0.693674,0.026431,partial_0.001
4,1.113832,0.579263,0.694733,0.69277,0.003503,partial_0.0001
5,0.579547,0.69945,0.818345,0.798093,0.127461,origin
6,0.902633,0.566764,0.678661,0.681659,0.000191,barycentre
7,1.037146,0.615288,0.728515,0.724199,0.087271,partial_0.01
8,1.039891,0.613875,0.699852,0.708954,0.028237,partial_0.001
9,1.037299,0.616844,0.699442,0.709677,0.003718,partial_0.0001


In [183]:
thresh=0.3
x_list = ['age','education-num']
methods=['origin','barycentre','partial'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(5):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label)
    
    print(projpost.postprocess('origin'))
    print(projpost.postprocess('barycentre'))
    for t in range(2,4):
        print(projpost.postprocess('partial',para=10**(-t)))

# report.to_csv(path+'/data/E3_postprocess_adult_'+str(pa)+'.csv',index=None)

         DI  f1 macro  f1 micro f1 weighted TV distance  method
0  0.471475  0.654216  0.813021    0.776355    0.130341  origin
         DI  f1 macro  f1 micro f1 weighted TV distance      method
0  0.645218  0.567431  0.685315     0.68513    0.000744  barycentre
         DI  f1 macro  f1 micro f1 weighted TV distance        method
0  0.541425  0.623109  0.803706    0.759091    0.085914  partial_0.01
         DI  f1 macro  f1 micro f1 weighted TV distance         method
0  0.552295  0.623134  0.803399    0.758986    0.026997  partial_0.001
         DI  f1 macro  f1 micro f1 weighted TV distance  method
0  0.531239  0.670878  0.814301    0.785454    0.129921  origin
         DI  f1 macro  f1 micro f1 weighted TV distance      method
0  1.078367  0.531828  0.646926    0.654245     0.00011  barycentre
         DI  f1 macro  f1 micro f1 weighted TV distance        method
0  0.550629  0.665578  0.813329    0.782802    0.083955  partial_0.01


KeyboardInterrupt: 

In [137]:
valpost = Projpostprocess(X_val,y_val,x_list,var_list,clf,K,e,'auto')
valpost.thresh

Optional threshold =  [0.1        0.18888889 0.27777778 0.36666667 0.45555556 0.54444444
 0.63333333 0.72222222 0.81111111 0.9       ]
Disparate Impact =  [0.66900341 0.65569828 0.65472977 0.65570698 0.6399922  0.6399922
 0.64740833 0.67114259 0.6775284  0.67792388]
f1 scores =  [0.66179706 0.66264053 0.66297696 0.66308919 0.66321851 0.66321851
 0.65267297 0.64056642 0.63902052 0.63649243]


0.9

In [78]:
methods=['origin','unconstrained','barycentre','partial','ROC'] # Place ROC in the end
report=pd.DataFrame(columns=['DI','f1 macro','f1 micro','f1 weighted','TV distance','method'])
for ignore in range(10):
    # train val test 4:2:4
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

    clf=RandomForestClassifier(max_depth=5).fit(X_train[:,0:var_dim],y_train)
    projpost = Projpostprocess(X_test,y_test,x_list,var_list,clf,K,e,thresh,favorable_label)
    for method in methods[:-1]:
        # report = pd.concat([report,projpost.postprocess(method,para=1e-2)], ignore_index=True)
        report = pd.concat([report,projpost.postprocess(method,para=1e-3)], ignore_index=True)

    ROCpost = ROCpostprocess(X_val,y_val,var_list,clf,favorable_label) # use validation set to train a ROC model
    report = pd.concat([report,ROCpost.postprocess(X_test,y_test,tv_origin=projpost.tv_origin)], ignore_index=True)

report.to_csv(path+'/data/report_postprocess_adult_'+str(pa)+'.csv',index=None)

KeyboardInterrupt: 

In [24]:
pa = 'race'
label_map = {1.0: '>50K', 0.0: '<=50K'}
privileged_groups = [{pa: 1}]
unprivileged_groups = [{pa: 0}]
if pa == 'sex':
    thresh=0.05
    protected_attribute_maps = [{1.0: 'Male', 0.0: 'Female'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['Male'],[1.0]], 
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps},
        # categorical_features=['workclass', 'marital-status', 'occupation', 'relationship', 'native-country']
        features_to_drop=['race','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
elif pa == 'race':
    thresh=0.1
    protected_attribute_maps = [{1.0: 'White', 0.0:'Non-white'}]
    cd = AdultDataset(protected_attribute_names=[pa],privileged_classes=[['White'],[1.0]],
        metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps}, #
        features_to_drop=['sex','fnlwgt','education','relationship',
                          'native-country','workclass','marital-status','occupation'])
    #,'workclass','marital-status','occupation','relationship',

# train,test = cd.split([0.6], shuffle=True) #len(test.instance_names) = 2057
var_list = cd.feature_names.copy()
var_list.remove(pa)
var_dim=len(var_list)

K=200
e=0.01
bins_capitalgain=[100,3500,7500,10000]
bins_capitalloss=[100,1600,1900,2200]

messydata=cd.convert_to_dataframe()[0]
messydata=messydata.rename(columns={pa:'S',cd.label_names[0]:'Y'})
messydata=messydata[(messydata['S']==1)|(messydata['S']==0)]
for col in var_list+['S','Y']:
    messydata[col]=messydata[col].astype('int64')
messydata['W']=cd.instance_weights
# project 0-100 to {0,1,...,5}
messydata['age']=np.floor((messydata['age'].to_numpy()-17)/15)
messydata['hours-per-week']=np.floor(messydata['hours-per-week'].to_numpy()/20)
messydata=categerise(messydata,'capital-gain',bins_capitalgain)
messydata=categerise(messydata,'capital-loss',bins_capitalloss)

X=messydata[var_list+['S','W']].to_numpy() # [X,S,W]
y=messydata['Y'].to_numpy() #[Y]
tv_dist=dict()
for x_name in var_list:
    x_range_single=list(pd.pivot_table(messydata,index=x_name,values=['W'])[('W')].index) 
    dist=rdata_analysis(messydata,x_range_single,x_name)
    tv_dist[x_name]=sum(abs(dist['x_0']-dist['x_1']))/2
x_list=[]
for key,val in tv_dist.items():
    if val>0.045:
        x_list+=[key]        
tv_dist

{'age': 0.04744857663969924,
 'education-num': 0.056762405582129784,
 'capital-gain': 0.021127950774052707,
 'capital-loss': 0.011363681253224836,
 'hours-per-week': 0.04445283428803036}